In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs
from gnss.tracking import tracking

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
!du -hs /mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310

1.3G	/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310



We are using data from the USRP front ends. This data contains *signed 4-bit complex* samples.


- L2:
    - 5MHz sampling frequency
    - 0Hz center frequency

- L1/L5:
    - 25MHz sampling
    - 0Hz center frequency

These files are large, so we should choose a reasonable buffer size for their data. 2 seconds worth of data would be:

In [4]:
5e6 * 2, 25e6 * 2

(10000000.0, 50000000.0)

which are 10 mega-samples and 50 mega-samples respectively.

In [5]:
f_center = 0.
f_samp = 5e6
filepath = '/mnt/gluster/by-location/ascension-island/a1/gpsl2/usrp5/20130307_184310'
source = sources.FileSignalSource(filepath, f_samp=f_samp, f_center=f_center, buffer_size=10e6, bit_depth=4, real=False)
source.load()
print('we have {0} seconds of data'.format(source.buffer_size / source.f_samp))

we have 2.0 seconds of data


In [24]:
svid = 21
signal = Signal.GPSL2(svid)

c_acq = coarse.CoarseAcquirer(source, 20e-3, 2)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [25]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp = c_acq.f_dopp
time = c_acq.time
snr = c_acq.snr
print(chip0, f_dopp, time, snr)

9834.148104 1100.0 0.0 21.2852301619


In [26]:
f_acq.acquire(signal, time, chip0, f_dopp)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)

9834.148104 1103.83367617 0.0


In [23]:
c_acq.acquire(signal)
chip1 = c_acq.chip
f_dopp1 = c_acq.f_dopp
time1 = c_acq.time
snr1 = c_acq.snr
print(chip1, f_dopp1, time1, snr1)

10216.876956 -1100.0 0.0923912 21.2912987945


In [ ]:
f_acq.acquire(signal, time1, 0, f_dopp)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)

In [19]:

f_chip - 1.023e6

0.8524999999208376

In [13]:
step = 2e-3
f_update = 1. / step
block_length = 2e-3
block_size = block_length * source.f_samp

dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
# pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 12, zeta=1.7, fs=f_update)

delay = .5
correlator = tracking.Correlator(chip_delays=[-delay, 0., delay])

output_buff_size = 1000
tracking_outputs = {
    'phase': {'size': output_buff_size, 'dtype': np.float},
    'chip': {'size': output_buff_size, 'dtype': np.float},
    'i_corr': {'size': output_buff_size, 'dtype': np.float},
    'q_corr': {'size': output_buff_size, 'dtype': np.float}
    }
output = outputs.TrackingOutputBuffer(**tracking_outputs)

In [30]:
time = 0.0
output.clear()

In [31]:
theta = 0.
chip = chip_out = chip0
while time < source.max_time() - block_length:
    code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
    early, prompt, late = code_corr
    chip_error = tracking.delay_discriminator(early, prompt, late)
    phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
    chip_error = dll_filter.f(chip_error)
    phase_error = pll_filter.f(phase_error)
    # error correction
    chip -= chip_error
    theta -= phase_error
    # carrier-aiding and propagation
    time += step
    f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
    chip += step * f_chip
    theta += step * (source.f_center + f_dopp) * 2. * np.pi  # TODO: why do both theta update lines work?
    # outputs
    chip_out = chip_out - chip_error
    output.push(**{'phase': phase_error, 'chip': chip_out, 'i_corr': np.real(prompt), 'q_corr': np.imag(prompt)})

In [32]:
output.plot()
plt.show()